In [1]:
import zipfile
import spacy

In [2]:
input_text = []
annotated = []
words_queue = []
files = set()
with zipfile.ZipFile('MACCROBAT2018.zip', 'r') as z:
    for filename in z.namelist():
        files.add(filename.split(".")[0])

    for filename in files:
        with z.open(filename + ".txt") as file:
            for line in file:
                sentence = line.decode('utf-8').strip()
                if sentence:
                    input_text.append(sentence)

with zipfile.ZipFile('MACCROBAT2020.zip', 'r') as z:
    for filename in z.namelist():
        files.add(filename.split(".")[0])

    for filename in files:
        with z.open(filename + ".txt") as file:
            for line in file:
                sentence = line.decode('utf-8').strip()
                if sentence:
                    input_text.append(sentence)

In [3]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def generate_detailed_tags(sentences):
    detailed_tags = []

    for index, sentence in enumerate(sentences):
        if index % 1000 == 0:
            print(index)

        # Process the sentence with spaCy
        doc = nlp(sentence)

        # Initialize BIO tags as "O" for all tokens
        bio_tags = ["O"] * len(doc)

        # Assign B- and I- tags based on entities
        for ent in doc.ents:
            start = ent.start
            end = ent.end

            # First token in the entity gets B- prefix
            bio_tags[start] = f"B-{ent.label_}"

            # Subsequent tokens in the entity get I- prefix
            for i in range(start + 1, end):
                bio_tags[i] = f"I-{ent.label_}"

        # Collect tokens, POS tags, and BIO tags
        tokens = [token.text for token in doc]
        pos_tags = [token.pos_ for token in doc]

        # Add to the output
        detailed_tags.append({
            "sentence": sentence,
            "tokens": tokens,
            "pos_tags": pos_tags,
            "ner_tags": bio_tags
        })

    return detailed_tags


In [4]:
# Generate detailed tags
detailed_tags_output = generate_detailed_tags(input_text)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [5]:
# Print output
for entry in detailed_tags_output[:5]:
    print(f"Sentence: {entry['sentence']}")
    print(f"Tokens: {entry['tokens']}")
    print(f"POS Tags: {entry['pos_tags']}")
    print(f"NER Tags: {entry['ner_tags']}")
    print()

Sentence: A 46-year-old Caucasian woman with type 2 diabetes mellitus and bipolar disorder presented to our emergency department with vague abdominal symptoms and vomiting.
Tokens: ['A', '46', '-', 'year', '-', 'old', 'Caucasian', 'woman', 'with', 'type', '2', 'diabetes', 'mellitus', 'and', 'bipolar', 'disorder', 'presented', 'to', 'our', 'emergency', 'department', 'with', 'vague', 'abdominal', 'symptoms', 'and', 'vomiting', '.']
POS Tags: ['DET', 'NUM', 'PUNCT', 'NOUN', 'PUNCT', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NUM', 'NOUN', 'ADJ', 'CCONJ', 'ADJ', 'NOUN', 'VERB', 'ADP', 'PRON', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT']
NER Tags: ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'B-NORP', 'O', 'O', 'O', 'B-CARDINAL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Sentence: Her pertinent history includes left below knee amputation and right toes amputation for complications secondary to diabetic neuropathy.
T

In [ ]:
import json

# Save to a JSON file
with open("tagged_medical_sentences.json", "w") as f:
    json.dump(detailed_tags_output, f, indent=4)

print("Output saved to 'tagged_medical_sentences.json'")